## Imports

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#from pandas.io.json import json_normalize
from dotenv import load_dotenv
import os
import re


## RIP to the code below an increasingly desperate attempt to scrape JavaScript as if it is HTML

In [29]:


redballoon_base_url = "https://www.redballoon.work/search?"

for i in range(2):
    print("fetching page " + str(i+1))
    rb_url = (redballoon_base_url + "postings%5Bpage%5D="+str(i+1)+"&radius=25")
    #print(rb_url)
    #rb_url="https://www.redballoon.work/search?postings%5Bpage%5D=1&radius=25"
    rb_html = requests.get(rb_url).text
    #print(rb_html)
    #print(len(rb_html))
    rb_soup = BeautifulSoup(rb_html)
  

    body = rb_soup.select('div', {'class': "raw-html-component"})
    
    print(f'result of select ||{body}||')

  
    #print(rb_soup.find_all('p'))
    #print(rb_soup.prettify())
    #print("title ", rb_soup.title)
    #print(rb_soup.find_all('meta'))


    #table = job_post.select("div.card.rounded-sm.bg-white-500.shadow-lg.p-6.mt-6.slow-fade-in.cursor-pointer.transistion.ease-in-out.duration-300.hover\:scale-103.w-full")
    #print(job_post)
    #item = rb_soup.select("div.card.rounded-sm.bg-white-500.shadow-lg.p-6.mt-6.slow-fade-in.cursor-pointer.transistion.ease-in-out.duration-300.hover\:scale-103.w-full")
    #for x in item:
     #   title = x.h4.text
      #  print(title)
    
    #for x in place_date:
        #date = x[-12:]
        #title = x[:-12]
        #state = x[-17:-15]



fetching page 1
result of select ||[<div id="__next"><div class="flex flex-col min-h-screen"><div class="page-content-wrapper grow bg-rebrand-gray-200" test-id="page-content-wrapper"></div><footer><div class="bg-navy-800 w-full flex justify-center font-roboto relative z-10 2xl:py-12 undefined"><div class="max-w-7xl w-full p-12 sm:p-24 flex flex-col lg:flex-row justify-between"><div class="flex flex-col"><div class="mb-6 border-t-[1px] border-navy-200 w-full"></div><a class="w-fit undefined" href="/"><span class=""><span class="text-navy-200 hover:text-navy-400 hover:border-b-navy-400 text-sm font-light hover:border-b-2 box-border">Home</span></span></a><a class="w-fit undefined" href="/contact" id="contact"><span class=""><span class="text-navy-200 hover:text-navy-400 hover:border-b-navy-400 text-sm font-light hover:border-b-2 box-border">Contact</span></span></a><a class="w-fit undefined" href="/about" id="about-us"><span class=""><span class="text-navy-200 hover:text-navy-400 hover:b

Hiding API KEy

In [30]:
import requests
import json
import os
load_dotenv
API_KEY = os.getenv("RB_API_KEY")

Credit here to Jeremy Singer-Vine who created the basis of my JavaScript scraper

Also a shout out to Harsha (TA) for helping me to normalize the large JSON dataset and minimise how much data made it into my data frame

In [31]:

df_content = []
for x in range(84):
  if x>0:
    payload = {
      "searches": [
        {
          "query_by": "title",
          "collection": "postings",
          "q": "*",
          "page": x
        }
      ]
    }
#hide API KEY
    params = {
      "x-typesense-api-key": {API_KEY},
    }

    data = json.dumps(payload)

    response = requests.post(
        "https://search.redballoon.work/multi_search",
        params=params,
        data=data
    )

    page_response = response.json()
    for hit in page_response['results'][0]['hits']:
      interested_variables = hit['document']
      df_content.append(interested_variables)
    guest_json = pd.json_normalize(df_content)

#print(guest_json)


In [32]:
df = pd.DataFrame(guest_json)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)
print(df.columns)

Index(['_geo', 'address1', 'address2', 'business_name', 'city', 'country_code',
       'id', 'job_roles', 'job_summary', 'job_type', 'pay_period',
       'pay_structure', 'postal_code', 'posted_datetime', 'qualifications',
       'region', 'remote_type', 'responsibilities', 'salary_from', 'salary_to',
       'tags', 'title'],
      dtype='object')


# Now to clean

First step: get the html out of my job_summary, qualifications and responsibilities using regex

In [33]:
html_pattern = re.compile('<.*?>')
html_pattern2 = re.compile('<.*(?=[^>]*\b\w+)')
df['job_summary']=df['job_summary'].astype(str)
df['job_summary']=df['job_summary'].apply(lambda x: re.sub(html_pattern,'',x))
df['job_summary']= df['job_summary'].apply(lambda x: re.sub(html_pattern2,'',x))
df['job_summary']=df['job_summary'].replace('&nbsp;','', regex = True).replace("&middot;",'', regex = True).replace('&bull;','', regex = True).replace('&rsquo;',"'", regex = True)

#qual
#keep getting expect str maybe entry is in html? so...
df['qualifications']=df['qualifications'].astype(str)
df['responsibilities']=df['responsibilities'].astype(str)

df['qualifications']= df['qualifications'].apply(lambda x: re.sub(html_pattern,'',x))
df['qualifications']= df['qualifications'].apply(lambda x: re.sub(html_pattern2,'',x))
df['qualifications']=df['qualifications'].replace('&nbsp;','', regex = True).replace("&middot;",'', regex = True).replace('&bull;','', regex = True).replace('&rsquo;',"'", regex = True)
#for responsibilities

df['responsibilities']= df['responsibilities'].apply(lambda x: re.sub(html_pattern,'',x))
df['responsibilities']= df['responsibilities'].apply(lambda x: re.sub(html_pattern2,'',x))
df['responsibilities']=df['responsibilities'].replace('&nbsp;','', regex = True).replace("&middot;",'', regex = True).replace('&bull;','', regex = True).replace('&rsquo;',"'", regex = True)

#df

In [34]:
#df.to_csv('proj2.csv', index = False)

In [35]:
df_remote = df['remote_type'].sort_values()
#print(len(df['remote_type']=='Hybrid'))
#df_remote

In [91]:
df_salary = df['salary_to'].median()
print(df_salary)
df_salary_lower = df['salary_from'].median()
print(df_salary_lower)

df_salary_avg_u = df['salary_to'].mean()
df_salary_avg_l = df['salary_from'].mean()

print(df_salary_avg_l, "-", df_salary_avg_u)

55000.0
45000.0
56808.14925773196 - 71289.22426804123


In [36]:
import matplotlib.pyplot as plt
#!pip install nltk==3.8.1


In [37]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
#!pip install scikit-learn


In [38]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
from nltk.corpus import stopwords
print(sklearn_stop_words)
#try and miss boring words found everywhere
additional_english_stop = ['www', 'things', 'does', 'x200b', 'amp',
                           'just', 'like', 'https', 'com', 'watch', 'want',
                           'says', 'say', 'did', 'this', 
                           'nan','time','including','000','00 000','00','ll','com','100','2023','50','500','00 00','10','30','40','monday fridayweekend']
new_stop_list = sklearn_stop_words.union(additional_english_stop)
print((new_stop_list))


frozenset({'besides', 'take', 'either', 'first', 'have', 'though', 'thus', 'sometimes', 'during', 'every', 'whole', 'why', 'ourselves', 'ever', 'whose', 'myself', 'nobody', 'them', 'there', 'few', 'anyhow', 'what', 'towards', 'although', 'keep', 'whence', 'any', 'whoever', 'whither', 'hereby', 'mine', 'together', 'same', 'beforehand', 'they', 'interest', 'give', 'herein', 'wherever', 'over', 'other', 'eleven', 'you', 'amoungst', 'name', 'these', 'put', 'go', 'from', 'wherein', 'when', 'by', 'each', 'six', 'own', 'their', 'becoming', 'such', 'enough', 'toward', 'afterwards', 'itself', 'anyway', 'the', 'am', 'much', 'otherwise', 're', 'being', 'elsewhere', 'fifty', 'would', 'co', 'somehow', 'within', 'eg', 'must', 'amongst', 'or', 'thin', 'than', 'mill', 'we', 'nor', 'therefore', 'meanwhile', 'yet', 'never', 'so', 'us', 'seemed', 'someone', 'for', 'his', 'that', 'hereafter', 'i', 'one', 'seem', 'while', 'all', 'alone', 'side', 'yours', 'two', 'throughout', 'cry', 'hundred', 'forty', 'thi

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
frequency_list =[]
tf_vectorizer = CountVectorizer(max_df=0.8, min_df=8,
    ngram_range = (1,2),
    binary=True,
    stop_words= list(new_stop_list))


In [40]:
term_frequencies = tf_vectorizer.fit_transform(df['job_summary'])

phrases_df = pd.DataFrame(data=tf_vectorizer.get_feature_names_out(),columns=['phrase'])
phrases_df['total_occurrences']=term_frequencies.sum(axis=0).T
phrases_df.sort_values(by='total_occurrences',ascending=False)
phrases_df.sort_values(by='total_occurrences',ascending=False).head(8000).to_csv('top_8000_overall_keywords_in.csv',index=False)
#phrases_df['phrase'].str.findall("election").sort_values()
#phrases_df['phrase']

/Users/julieiles/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fridayweekend', 'monday'] not in stop_words.
  warnings.warn(


In [41]:
political_words = {"election",'desantis','political','politics','campaign','republican','biden','canvassers','canvas','electoral','presidential candidate','states turning','running','run','vote','federal races','party','governor','don','donald','trump','cycle','grassroots','campaigns'}
election_words = {" election","desantis","canvas","canvassers","presidential candidate","donald trump","trump","governor"}
trump_words = {' Trump'}
desantis = "desantis"
biden_words = {' biden',"Biden"}
mask = df['job_summary'].str.contains('|'.join(political_words))
political_terms_df = df[mask]
political_terms_df.to_csv('political_ads_again.csv',index=False)

just_desantis_df = df[df['job_summary'].str.contains(desantis)]
just_trump_df = df[df['job_summary'].str.contains('|'.join(trump_words))]
just_biden_df = df[df['job_summary'].str.contains('|'.join(biden_words))]
election_mention = df[df['job_summary'].str.contains('|'.join(election_words))]
#just_trump_df.to_csv('trump_mention.csv',index=False)
#just_desantis_df.to_csv('desantis_mention.csv',index=False)
#just_biden_df.to_csv('biden_mention.csv',index=False)

#election_mention.to_csv('2024_election_mention.csv',index=False)

#just_biden_df.to_csv('biden_mention.csv',index=False)
#election_mention.to_csv('2024_election_mention.csv',index=False)




In [62]:
gun_words={"self-defense","armed","carrying association","gun","firearms","gun","guns","2nd"}
christian_words = {"biblical","bible","faith-based","faith","christian","Christian","God"}
activist_words={"grassroots","activist","activism"}
medical_words = {"hippa", "vaccine","medical liberty","wellness","natural medicine","vaccine-related injury"}
dog_whistle_words = {"natural born","born right","american values","inner city", "welfare queen"}

christian_jobs_df = df[df['job_summary'].str.contains('|'.join(christian_words))]
christian_jobs_df.to_csv('christian_jobs.csv',index=False)
christian_jobs_df2 = pd.read_csv('godjobs:locs.csv')
df_christian_lat = christian_jobs_df2['Lon/lat'].apply(lambda x: str(x).strip('[]').split(',')[0]) if len(str(x).split(','))>=1 else None

df_christian_lon = christian_jobs_df2['Lon/lat'].apply(lambda x: '-' + str(x).split('-')[-1].strip('[]') if '-' in str(x) else str(x).split(',')[1].strip('[]'))
print(df_christian_lon)




IndexError: list index out of range

In [93]:
vax = {"vaccine-injury"}
df_construction = df[df['job_summary'].str.contains('|'.join(vax))]

df_construction.to_csv("vax_jobs.csv", index = False)

        

